In [61]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder

### Test data

In [306]:
df_test = pd.read_csv('../../ml-data/cyberattack_detection/1_raw_data/A2_2/test_data_with_labels', sep=','
                       ,names=['Timestamp','Duration','Protocol','SrcIPAddress','SrcPort','Direction',
                               'DestIPAddress','DestPort','State','SrcTypeService','DestTypeService',
                               'TotalPackets','TotalBytes','SourceBytes','Label'])

In [304]:
df_test_filtered = df_test[~df_test.isin([np.nan, np.inf, -np.inf]).any(1)]

In [305]:
df_test_filtered.loc[df_test_filtered['SrcIPAddress']=="147.101.94.182"].head()

,Timestamp,Duration,Protocol,SrcIPAddress,SrcPort,Direction,DestIPAddress,DestPort,State,SrcTypeService,DestTypeService,TotalPackets,TotalBytes,SourceBytes,Label
30,2011/08/17 11:57:59.581994,0.000255,udp,147.101.94.182,1025,<->,147.32.90.101,53,CON,0.0,0.0,2,203,66,flow=From-Botnet-V50-3-UDP-DNS
31,2011/08/17 11:58:02.013275,0.020982,udp,147.101.94.182,1025,<->,147.32.90.101,53,CON,0.0,0.0,2,569,83,flow=From-Botnet-V50-3-UDP-DNS
32,2011/08/17 11:58:02.022016,0.037351,tcp,147.101.94.182,1027,->,74.125.242.192,80,SRPA_SPA,0.0,0.0,7,913,607,flow=From-Botnet-V50-3-TCP-HTTP-Google-Net-Est...
34,2011/08/17 11:58:04.765808,0.000418,udp,147.101.94.182,1025,<->,147.32.90.101,53,CON,0.0,0.0,2,477,77,flow=From-Botnet-V50-3-UDP-DNS
31784,2011/08/17 14:15:02.224660,0.508984,udp,147.101.94.182,1025,<->,147.32.90.101,53,CON,0.0,0.0,2,211,70,flow=From-Botnet-V50-3-UDP-DNS


In [65]:
50*15402

770100

In [307]:
df_test.at[30, 'Duration'] =  896600 + 0.000255
df_test.at[30, 'TotalPackets'] = 896600 + 2
df_test.at[30, 'TotalBytes'] = 896600 + 203
df_test.at[30, 'SourceBytes'] = 896600 + 66

In [236]:
df_test.loc[df_test['SrcIPAddress']=="147.101.94.182"].head()

,Timestamp,Duration,Protocol,SrcIPAddress,SrcPort,Direction,DestIPAddress,DestPort,State,SrcTypeService,DestTypeService,TotalPackets,TotalBytes,SourceBytes,Label
30,2011/08/17 11:57:59.581994,898000.000000,udp,147.101.94.182,1025,<->,147.32.90.101,53,CON,0.0,0.0,898000,898000,898000,flow=From-Botnet-V50-3-UDP-DNS
31,2011/08/17 11:58:02.013275,0.020982,udp,147.101.94.182,1025,<->,147.32.90.101,53,CON,0.0,0.0,2,569,83,flow=From-Botnet-V50-3-UDP-DNS
32,2011/08/17 11:58:02.022016,0.037351,tcp,147.101.94.182,1027,->,74.125.242.192,80,SRPA_SPA,0.0,0.0,7,913,607,flow=From-Botnet-V50-3-TCP-HTTP-Google-Net-Est...
33,2011/08/17 11:58:04.035463,0.079349,udp,147.101.94.182,123,->,65.55.66.30,123,INT,0.0,NaN,2,185,187,flow=From-Botnet-V50-3-UDP-Attempt
34,2011/08/17 11:58:04.765808,0.000418,udp,147.101.94.182,1025,<->,147.32.90.101,53,CON,0.0,0.0,2,477,77,flow=From-Botnet-V50-3-UDP-DNS


### Traffic perturbation

In [113]:
df_test.to_csv('../../ml-data/cyberattack_detection/1_raw_data/A2_2/test_data_with_labels_perturbed',
                         sep=',', index=False, header=None )

### Reading perturbed file

In [114]:
df_test = pd.read_csv('../../ml-data/cyberattack_detection/1_raw_data/A2_2/test_data_with_labels_perturbed', sep=','
                       ,names=['Timestamp','Duration','Protocol','SrcIPAddress','SrcPort','Direction',
                               'DestIPAddress','DestPort','State','SrcTypeService','DestTypeService',
                               'TotalPackets','TotalBytes','SourceBytes','Label'])

In [290]:
#df_test.loc[df_test['SrcIPAddress'] == "147.101.94.182"]

In [308]:
new_label_test = pd.DataFrame(data=np.where(df_test['Label'].str.contains("Botnet",case=False), 1, 0),columns=["Botnet"])

In [309]:
df_test_sel = df_test[['SrcIPAddress','DestIPAddress','Direction','Duration','TotalPackets','TotalBytes','SourceBytes']]

In [310]:
df_test_sel = pd.concat([df_test_sel,new_label_test], axis=1)

In [311]:
df_test_sel.head()

,SrcIPAddress,DestIPAddress,Direction,Duration,TotalPackets,TotalBytes,SourceBytes,Botnet
0,147.143.92.183,147.32.88.101,<->,0.000281,2,203,61,1
1,147.143.92.183,74.125.240.193,->,0.050073,7,861,659,1
2,147.143.92.183,147.32.88.101,<->,0.000455,3,575,87,1
3,147.143.92.183,147.32.88.101,<->,0.000402,3,484,75,1
4,147.143.92.183,65.55.64.32,->,3.049152,3,188,183,1


In [312]:
df_test_sel['PacketsBySec'] = df_test_sel['TotalPackets']/df_test_sel['Duration']
df_test_sel['SourceBytesBySec'] = df_test_sel['SourceBytes']/df_test_sel['Duration']
df_test_sel['TotalBytesBySec'] = df_test_sel['TotalBytes']/df_test_sel['Duration']

In [313]:
df_test_filtered = df_test_sel[~df_test_sel.isin([np.nan, np.inf, -np.inf]).any(1)]

In [314]:
df_group_test = df_test_filtered.groupby(['SrcIPAddress']).mean()

In [315]:
df_group_test['Botnet'].value_counts()

0.00000    29
1.00000    26
0.99994     1
0.99985     1
Name: Botnet, dtype: int64

In [316]:
df_group_test.loc[(df_group_test['Botnet'] > 0.5), 'Botnet'] = 1.0

In [317]:
df_group_test.head()

,Duration,TotalPackets,TotalBytes,SourceBytes,Botnet,PacketsBySec,SourceBytesBySec,TotalBytesBySec
SrcIPAddress,,,,,,,,
122.226.12.150,0.068330,2.099315,125.989726,68.424658,0.0,10164.267421,313196.425928,609901.995847
147.101.94.182,87.038085,56.736226,2669.486505,1010.705275,1.0,1495.671372,43775.022442,211851.928670
147.113.94.198,35.870162,13.585749,9117.484682,1401.067273,1.0,1661.905285,48534.566173,237852.833934
147.114.98.192,13.462622,7.996686,4687.332367,504.842604,1.0,2198.629757,64127.241159,318626.536661
147.122.97.192,15.085764,16.219029,11824.440682,1431.609252,1.0,1780.671385,52146.694391,257579.859516


In [318]:
df_group_test.to_csv('../../ml-data/cyberattack_detection/2_processed_data/A2_test_data_by_ip_perturbed.csv', sep=',')

End.